In [ ]:
!pip install langchain
!pip install pinecone-client
!pip install transformers
!pip install ctransformers
!pip install langchain pinecone-client transformers ctransformers


In [ ]:
!pip install -U langchain-community


In [ ]:
from langchain import PromptTemplate
from langchain.chains import RetrievalQA
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import Pinecone
import pinecone
from langchain.document_loaders import PyPDFLoader, DirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.prompts import PromptTemplate
from langchain.llms import CTransformers


In [2]:
#Extract data from the PDF
def load_pdf(data):
    loader = DirectoryLoader(data,
                    glob="*.pdf",
                    loader_cls=PyPDFLoader)
    
    documents = loader.load()

    return documents

In [3]:
pip install pypdf

Note: you may need to restart the kernel to use updated packages.


In [4]:
extracted_data = load_pdf("data/")

In [ ]:
extracted_data

In [6]:
#Create text chunks
def text_split(extracted_data):
    text_splitter = RecursiveCharacterTextSplitter(chunk_size = 500, chunk_overlap = 20)
    text_chunks = text_splitter.split_documents(extracted_data)

    return text_chunks

In [7]:
text_chunks = text_split(extracted_data)
print("length of my chunk:", len(text_chunks))

length of my chunk: 7020


In [ ]:
#text_chunks

In [8]:
#download embedding model
def download_hugging_face_embeddings():
    embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
    return embeddings

In [ ]:
pip install sentence-transformers

In [9]:
embeddings = download_hugging_face_embeddings()

c:\Users\Dell\anaconda3\envs\mchat\Lib\site-packages\langchain_core\_api\deprecation.py:139: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 0.3.0. An updated version of the class exists in the langchain-huggingface package and should be used instead. To use it run `pip install -U langchain-huggingface` and import as `from langchain_huggingface import HuggingFaceEmbeddings`.
  warn_deprecated(


In [ ]:
embeddings

In [11]:
query_result = embeddings.embed_query("Hello world")
print("Length", len(query_result))

Length 384


In [ ]:
query_result

In [35]:
from pinecone import Pinecone, ServerlessSpec

In [ ]:
from langchain.vectorstores import Pinecone as PC
from langchain_pinecone import Pinecone

In [ ]:
PINECONE_API_KEY = ""

In [31]:
pc = Pinecone(api_key=PINECONE_API_KEY)

In [37]:
#Initializing the Pinecone
pc.create_index(
    name="medical-chatbot-1",
    dimension=384, 
    metric="cosine", 
    spec=ServerlessSpec(
        cloud="aws",
        region="us-east-1"
    ) 
)

In [ ]:
pip install langchain_pinecone

In [51]:
from langchain_pinecone import PineconeVectorStore

In [56]:
import os

In [57]:
os.environ['PINECONE_API_KEY'] = ''


In [58]:
index_name = "medical-chatbot-1"
docs_chunks = [t.page_content for t in text_chunks]
vectorstore = PineconeVectorStore.from_texts(
    docs_chunks,
    index_name=index_name,
    embedding=embeddings
)

In [59]:
query = "What are Allergies"

docs=vectorstore.similarity_search(query, k=3)

print("Result", docs)

Result [Document(page_content="GALE ENCYCLOPEDIA OF MEDICINE 2 117Allergies\nAllergic rhinitis is commonly triggered by\nexposure to household dust, animal fur,or pollen. The foreign substance thattriggers an allergic reaction is calledan allergen.\nThe presence of an allergen causes the\nbody's lymphocytes to begin producingIgE antibodies. The lymphocytes of an allergy sufferer produce an unusuallylarge amount of IgE.\nIgE molecules attach to mast\ncells, which contain histamine.HistaminePollen grains\nLymphocyte\nFIRST EXPOSURE"), Document(page_content='allergens are the following:\n• plant pollens\n• animal fur and dander\n• body parts from house mites (microscopic creatures\nfound in all houses)\n• house dust• mold spores• cigarette smoke• solvents• cleaners\nCommon food allergens include the following:\n• nuts, especially peanuts, walnuts, and brazil nuts\n• fish, mollusks, and shellfish• eggs• wheat• milk• food additives and preservatives\nThe following types of drugs commonly ca

In [60]:
prompt_template="""
Use the following pieces of information to answer the user's question.
If you don't know the answer, just say that you don't know, don't try to make up an answer.

Context: {context}
Question: {question}

Only return the helpful answer below and nothing else.
Helpful answer:
"""

In [61]:
PROMPT=PromptTemplate(template=prompt_template, input_variables=["context", "question"])
chain_type_kwargs={"prompt": PROMPT}

In [62]:
llm=CTransformers(model="model/llama-2-7b-chat.ggmlv3.q4_0.bin",
                  model_type="llama",
                  config={'max_new_tokens':512,
                          'temperature':0.8})

In [64]:
qa=RetrievalQA.from_chain_type(
    llm=llm, 
    chain_type="stuff", 
    retriever=vectorstore.as_retriever(search_kwargs={'k': 2}),
    return_source_documents=True, 
    chain_type_kwargs=chain_type_kwargs)

In [65]:
while True:
    user_input=input(f"Input Prompt:")
    result=qa({"query": user_input})
    print("Response : ", result["result"])

c:\Users\Dell\anaconda3\envs\mchat\Lib\site-packages\langchain_core\_api\deprecation.py:139: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 0.3.0. Use invoke instead.
  warn_deprecated(


Response :  Acne is a common skin disease characterized by pimples on the face, chest, and back. It occurs when the pores of the skin become clogged with oil, dead skin cells, and bacteria.
Response :  The concept of "right" and "want" can be complex and context-dependent, as different people may have varying moral, cultural, or personal beliefs and values. In general, what is considered right or wrong can depend on a variety of factors such as social norms, laws, ethical principles, and individual experiences.
In the context of doing what one wants, it's important to recognize that doing something just because it feels good in the moment without considering long-term consequences can lead to negative outcomes. For example, engaging in risky or irresponsible behavior can have serious consequences on one's well-being and relationships.
On the other hand, prioritizing long-term goals and values over short-term desires can also be challenging, as it may require sacrifices and hard work. H

KeyboardInterrupt: 